In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert  # para la envolvente
import pywt
from scipy.linalg import svd

Parámetros sacados del paper

In [ ]:
# Parameters
fs = 1e9  # Sampling frequency: 1 GHz
duration = 2e-6  # 2 microseconds
t = np.linspace(0, duration, int(fs * duration))

# Echo signal parameters (matching paper values)
echo_params = [
    {'A': 10, 'alpha': 300e12, 'f': 100e6, 'tau': 0.4e-6},
    {'A': 8, 'alpha': 300e12, 'f': 100e6, 'tau': 0.6e-6}
]

In [ ]:
def generate_echo(t, A, alpha, f, tau):
    return A * np.exp(-alpha * (t - tau)**2) * np.cos(2 * np.pi * f * (t - tau))

# Add noise with desired SNR
def add_noise(signal, snr_db):
    signal_power = np.mean(signal**2)
    snr_linear = 10**(snr_db / 10)
    noise_power = signal_power / snr_linear
    noise = np.random.normal(0, np.sqrt(noise_power), size=signal.shape)
    return signal + noise


In [ ]:
def wavelet_soft_threshold_denoise(signal, wavelet='db4', level=None, mode='soft', sigma=None):
    """
    Denoise a 1D signal using wavelet soft thresholding (WST).

    Parameters:
    - signal: 1D numpy array, noisy input signal.
    - wavelet: string, wavelet name (e.g., 'db4').
    - level: int, decomposition level. If None, uses max level.
    - mode: 'soft' or 'hard' thresholding.
    - sigma: float, estimated noise standard deviation. If None, estimated from median absolute deviation.

    Returns:
    - denoised: 1D numpy array, denoised signal.
    """
    # Decompose
    coeffs = pywt.wavedec(signal, wavelet=wavelet, level=level)

    # Estimate noise sigma from detail coefficients at highest level
    if sigma is None:
        detail_coeffs = coeffs[-1]
        sigma = np.median(np.abs(detail_coeffs)) / 0.6745

    # Universal threshold
    uthresh = sigma * np.sqrt(2 * np.log(len(signal)))

    # Threshold detail coefficients
    denoised_coeffs = [coeffs[0]]  # approximation unchanged
    for c in coeffs[1:]:
        denoised_coeffs.append(pywt.threshold(c, uthresh, mode=mode))

    # Reconstruct signal
    denoised = pywt.waverec(denoised_coeffs, wavelet=wavelet)
    # Trim to original length
    return denoised[: signal.shape[0]]


In [ ]:
def gst_svd_denoise(signal, fs, a=1.0, b=1.0, energy_thresh=0.8):
    """
    Denoise a 1D signal using Generalized S-transform (GST) + SVD method.

    Parameters:
    - signal: 1D numpy array, noisy input signal.
    - fs: float, sampling frequency (Hz).
    - a, b: floats, Gaussian window parameters for GST.
    - energy_thresh: float in (0,1), cumulative energy threshold for singular values.

    Returns:
    - denoised: 1D numpy array, denoised signal.
    """
    # print timming times for each step

    import time
    start_time = time.time()

    N = len(signal)
    t = np.arange(N) / fs
    freqs = np.fft.fftfreq(N, d=1/fs)
    Xf = np.fft.fft(signal)

    print(f"FFT computed in {time.time() - start_time:.4f} seconds")
    start_time = time.time()

    # Compute GST matrix S (complex)
    S = np.zeros((N, N), dtype=complex)
    for p, fp in enumerate(freqs):
        window = np.exp(-2 * (np.pi**2) * (a * t)**2 * (b * fp)**2)
        S[p, :] = np.fft.ifft(Xf * window * np.exp(2j * np.pi * fp * t))

    print(f"GST matrix computed in {time.time() - start_time:.4f} seconds")
    start_time = time.time()

    # SVD on S
    U, s_vals, Vh = svd(S, full_matrices=False)
    # Energy-based threshold on singular values
    energy = np.cumsum(s_vals**2) / np.sum(s_vals**2)
    k = np.searchsorted(energy, energy_thresh) + 1
    s_vals[k:] = 0

    print(f"SVD computed in {time.time() - start_time:.4f} seconds")
    start_time = time.time()

    # Reconstruct denoised S and inverse transform
    S_denoised = U @ np.diag(s_vals) @ Vh
    denoised = np.real(np.mean(S_denoised, axis=0))

    print(f"Denoised signal reconstructed in {time.time() - start_time:.4f} seconds")

    start_time = time.time()

    # Ensure output has same amplitude as input
    denoised *= np.linalg.norm(signal) / np.linalg.norm(denoised)

    print(f"Amplitude normalization completed in {time.time() - start_time:.4f} seconds")
    return denoised

In [ ]:
clean_signal = sum(generate_echo(t, **p) for p in echo_params)

snrs = [25, 15, 5]
noisy_signals = [add_noise(clean_signal, snr) for snr in snrs]

In [ ]:
# Plotting
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(t * 1e6, clean_signal)
plt.title("Clean Signal")
plt.xlabel("Time (μs)")
plt.xticks(np.linspace(0, duration, 5) * 1e6)

for i, snr in enumerate(snrs):
    plt.subplot(2, 2, i+2)
    plt.plot(t * 1e6, noisy_signals[i])
    plt.title(f"Noisy Signal (SNR = {snr} dB)")
    plt.xlabel("Time (μs)")

plt.tight_layout()
plt.show()



In [ ]:
denoised_wavelet = [wavelet_soft_threshold_denoise(signal, wavelet='db4', mode='soft') for signal in noisy_signals]

In [ ]:
denoised_gst_svd = [gst_svd_denoise(signal, fs) for signal in noisy_signals]

In [ ]:
# Plot denoised signals
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(t * 1e6, clean_signal)
plt.title("Clean Signal")
plt.xlabel("Time (μs)")
plt.xticks(np.linspace(0, duration, 5) * 1e6)
plt.legend(['Clean Signal'])
for i, snr in enumerate(snrs):
    plt.subplot(2, 2, i+2)
    plt.plot(t * 1e6, denoised_wavelet[i], label='Wavelet Denoising', color='tab:orange', alpha=0.5)
    plt.plot(t * 1e6, denoised_gst_svd[i], label='GST + SVD Denoising', color='tab:green', alpha=0.5)
    plt.title(f"Denoised Signal (SNR = {snr} dB)")
    plt.xlabel("Time (μs)")
    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Compute error metrics
def compute_metrics(clean, denoised):
    mse = np.mean((clean - denoised) ** 2)
    rmse = np.sqrt(mse)
    snr = 10 * np.log10(np.var(clean) / mse)
    return mse, rmse, snr
metrics_wavelet = [compute_metrics(clean_signal, denoised) for denoised in denoised_wavelet]
metrics_gst_svd = [compute_metrics(clean_signal, denoised) for denoised in denoised_gst_svd]

In [ ]:
# plot metrics
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.bar([f'SNR={snr} dB' for snr in snrs], [m[0] for m in metrics_wavelet], label='Wavelet Denoising', alpha=0.7)
plt.bar([f'SNR={snr} dB' for snr in snrs], [m[0] for m in metrics_gst_svd], label='GST + SVD Denoising', alpha=0.7, color='tab:green', bottom=[m[0] for m in metrics_wavelet])
plt.title("MSE of Denoised Signals")
plt.xlabel("SNR")
plt.ylabel("MSE")
plt.legend()
plt.subplot(1, 2, 2)
plt.bar([f'SNR={snr} dB' for snr in snrs], [m[1] for m in metrics_wavelet], label='Wavelet Denoising', alpha=0.7)
plt.bar([f'SNR={snr} dB' for snr in snrs], [m[1] for m in metrics_gst_svd], label='GST + SVD Denoising', alpha=0.7, color='tab:green', bottom=[m[1] for m in metrics_wavelet])
plt.title("RMSE of Denoised Signals")
plt.xlabel("SNR")
plt.ylabel("RMSE")
plt.legend()
plt.tight_layout()
plt.show()

Hacemos escaneo 2D de la oblea

In [ ]:
# Definimos el tamaño del escaneo
Nx, Ny = 100, 100
pixel_size = 0.1e-3  # 0.1 mm por píxel

# Creamos un “mapa de profundidad” de la oblea
depth_map = np.zeros((Nx, Ny))
for i in range(Nx):
    for j in range(Ny):
        # Si estamos en una ranura (digamos j entre 30 y 50)
        if i % 4 == 0:
            depth_map[i, j] = 0.15e-3  # 150 μm
        else:
            depth_map[i, j] = 0.5e-3   # resto de la oblea


In [ ]:
cscan = np.zeros((Nx, Ny))

for i in range(Nx):
    for j in range(Ny):
        # Ajustamos tiempo de llegada según la profundidad local
        local_tau = depth_map[i, j] / 1500  # v ~1500 m/s
        params = [
            {**echo_params[0], 'tau': local_tau},
            {**echo_params[1], 'tau': local_tau + 0.2e-6}
        ]
        # Generamos la señal limpia
        clean = sum(generate_echo(t, **p) for p in params)
        # Añadimos ruido (ej. SNR 10 dB)
        noisy = add_noise(clean, snr_db=120) # Por ahora, SNR muy alto
        # TODO: Implementar el procesamiento de la señal (sacarle el ruido)

        # 1) Extraemos la envolvente con Hilbert
        env = np.abs(hilbert(noisy))
        # 2) Tomamos el valor pico de la envolvente
        cscan[i, j] = np.max(env)

In [ ]:

# Para visualizar en escala real
x = np.arange(Nx) * pixel_size * 1e3  # en mm
y = np.arange(Ny) * pixel_size * 1e3  # en mm

plt.figure(figsize=(6,5))
plt.imshow(cscan.T, 
           extent=[x.min(), x.max(), y.min(), y.max()],
           origin='lower')
plt.xlabel('Posición X (mm)')
plt.ylabel('Posición Y (mm)')
plt.title('C-scan (amplitud pico de la envolvente)')
plt.colorbar(label='Amplitud')
plt.tight_layout()
plt.show()

Hacer script para dada una imagen, generar las señales con sus ruidos